# Generalizations of Max-Flow

The purpose of this assignment is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize yourself with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$ (see also Problem 1 in hw3t). In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems we discussed so far: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Question 1

Answer Problem 1 in hw4-theoretical.

### Question 2

To implement your reduction from Problem 1 in hw4t, you will work with some standard benchmark instances for min-cost flow included in the hw4p folder on canvas. The format of the data is described in the Info file. You are to read in the graph from the data file in a form that can be solved using NetworkX's `min_cost_flow` function. Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities. In forming the graph, you need to implement your reduction from the previous question and form a `DiGraph` instance, because the `min_cost_flow` function cannot handle multi-edges, even though the package offers `MultiDiGraph` objects.

In [1]:
import networkx as nx

def create_graph(infile):
    """Creates a directed graph as specified by the input file. Edges are annotated with 'capacity'
    and 'weight' attributes, and nodes are annotated with 'demand' attributes.
    
    Args:
        infile: the input file using the format to specify a min-cost flow problem.
        
    Returns:
        A directed graph (but not a multi-graph) with edges annotated with 'capacity' and 'weight' attributes
        and nodes annotated with 'demand' attributes.
    """
    G = nx.DiGraph()
    file1 = open(infile,"r+")  
    content = file1.read()
    lines = content.split("\n")
    i = 0
    for line in lines:
        if (line == ""):
            continue
        elif (line[0] == "c"):
            continue
        elif (line[0] == 'p'):
            continue
        elif (line[0] == "n"):
            temp = line.split(" ")
            G.add_node(temp[1])
            G.nodes[temp[1]]['demand'] = int(temp[2])
        elif (line[0] == "a"):
            temp = line.split(" ")
            med_node = temp[2] + "_" + temp[1] + "_" + str(i)
            i += 1
            G.add_node(med_node)
            G.nodes[med_node]['demand'] = 0
            if (G.has_node(temp[1]) == False):
                G.add_node(temp[1])
                G.nodes[temp[1]]['demand'] = 0
            if (G.has_node(temp[2]) == False):
                G.add_node(temp[2])
                G.nodes[temp[2]]['demand'] = 0
            G.add_edge(temp[1], med_node) 
            G.add_edge(med_node, temp[2])
            if(int(temp[4]) < 0):
                G.edges[temp[1], med_node]['capacity'] = (float("inf"))
            else:
                G.edges[temp[1], med_node]['capacity'] = int(temp[4])
            G.edges[temp[1], med_node]['weight'] = int(temp[5])
            G.edges[med_node, temp[2]]['capacity'] = int(temp[4])
            G.edges[med_node, temp[2]]['weight'] = 0                   
        else:
            continue
    return G

The following will check that your code outputs the correct min cost flow values on several test instances.

In [2]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

print("Correct value for _40 instance:", nx.flow.min_cost_flow_cost(G_40) == 52099553858)
print("Correct value for _6830 instance:", nx.flow.min_cost_flow_cost(G_6830) == 299390431788)
print("Correct value for _176280 instance:", nx.flow.min_cost_flow_cost(G_176280) == 510585093810)

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

Instead of using special-purpose min-cost flow solvers, you will now formulate the problems as linear programs and use general-purpose LP solvers to find the solutions.

### Question 3

Implement the following function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).

In [3]:
import pulp

def lp_flow_value(G):
    """Computes the value of the minimum cost flow by formulating and solving
    the problem as an LP.
    
    Args:
        G: a directed graph with edges annotated with 'capacity' and 'weight'
            attrbutes, and nodes annotated with 'demand' attributes.
  
    Returns:
        The value of the minimum cost flow.
    """    
    model = pulp.LpProblem("Minimum Cost Flow", pulp.LpMinimize)
      
    var = pulp.LpVariable.dicts("Flow",list(G.edges()),None,None,pulp.LpInteger)
    for (s1, s2) in G.edges():
        if(G.edges[s1,s2]['capacity'] != (float("inf"))):
            var[(s1,s2)].bounds(0 , G.edges[s1, s2]['capacity'])
    
    model += pulp.lpSum([var[(s1,s2)] * G.edges[s1, s2]['weight'] for (s1, s2) in G.edges()])
    
    for n in list(G.nodes()):
        model += (pulp.lpSum([var[(i,j)] for (i,j) in list(G.edges()) if j == n]) == 
             G.nodes[n]['demand'] + pulp.lpSum([var[(i,j)] for (i,j) in list(G.edges())  if i == n]))
    model.solve()
    return pulp.value(model.objective)

The following will check that the LP finds the same optimal values as previously.

In [4]:
print("Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858)
print("Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788)
print("Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810)

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
